In [208]:
#inserting libraries for the prject

import random #random forest generation
import numpy as np #vectorized computation
import pandas as pd #processing data to dataframes
import requests
import matplotlib.pyplot as plt #visualizing data via plotting
%matplotlib inline
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans #importing KMeans
from sklearn.datasets.samples_generator import make_blobs
import bs4 as bs
import urllib.request
import matplotlib.cm as cm
import matplotlib.colors as colors
print("All libraries imported")

All libraries imported


# **Part - Project 1 : Text Scrapping with Beautiful Soup** <br>
1) Start by creating a new Notebook for this assignment <br>
2) Scrape the following Wikipedia page in order to obtain the data <br>
3) Create the dataframe according to the bullets

**Obtaining data from the following Wikipedia page**

In [209]:
#The dataframe must contain three columns: Postalcode, Borough, Neighborhood

source = urllib.request.urlopen("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").read()
soup = bs.BeautifulSoup(source,"html.parser")

table = soup.find("table")
table_rows = table.find_all("tr")

l = []
for tr in table_rows:
    td = tr.find_all("td")
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        l.append(row)

In [210]:
df = pd.DataFrame(l, columns=["Postal Code","Borough","Neighborhood"])
df.head(10)

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,None
1,M2A,Not assigned,None
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,None
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,"Malvern, Rouge"


**Removing the boroughs with "Not assigned" label**

In [211]:
#Process the cells that have assigned boroughs. Ignore the remaining boroughs with "Not assigned"
df = df[df.Borough != "Not assigned"]
df.head(10)

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [212]:
#Gather all neighborhouds in each one borough
df = df.groupby(["Postal Code", "Borough"]).agg(", ".join)
df = df.reset_index()
df.head(10)

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


**Assigning the "not assigned" neighbourhoods with the same name as their boroughs**

In [213]:
#"Not assigned" neighborhoods will be the same as their borough
df.loc[df['Neighborhood']=="Not assigned", ["Neighborhood"]] = "Queen\'s Park'"
df.head(10)

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


**Checking the shape of the dataframe**

In [214]:
#Use the .shape method to print the number of rows of the dataframe
df.shape

(103, 3)

# Part - Project 2 : Finding the latitude and the longitude coordinates of each neighborhood

Geocoder was not working, so I used the link to download the csv file

In [215]:
#I download the .csv file from the link http://cocl.us/Geospatial_data
df_coord = pd.read_csv("Geospatial_Coordinates.csv")
Data_Location = pd.merge(df, df_coord, on = "Postal Code")

**Checking the data**

In [216]:
Data_Location.head(10)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


**Creating a new dataframe which contains only the borough "North York"**

In [217]:
Toronto_data = Data_Location[Data_Location["Borough"] == "Downtown Toronto"].reset_index(drop=True)
Toronto_data.head(10)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
1,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
5,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
6,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
7,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
9,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752


# Part - Project 3 : Exploring Toronto map

### First task is to generate Toronto Map and plotting North York neigborhoods

In [218]:
import folium

TorLat = 43.651070 #This is the Latitude
TorLong = -79.347015 #This is the Longitude
d_toronto = folium.Map(location=[TorLat,TorLong], zoom_start=10) #call Folium to generate the map
for lat, lng, borough, neighbourhood in zip(Toronto_data["Latitude"], Toronto_data["Longitude"], Toronto_data["Borough"], Toronto_data["Neighborhood"]):
    label = "{}, {}".format(Toronto_data,borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color="blue",
        fill=True,
        fill_color="#3186cc",
        fill_opacity=0.6,
        parse_html=False).add_to(d_toronto)

d_toronto

## Inserting the information from Foursquare

In [219]:
CLIENT_ID="OYFDUQVFQKXRFH4QQLRJUOE32CG22SNSF223SV23GBCP4D00"
CLIENT_SECRET = "Q22FWYTYWJJ4BYDR1WCXW11NJLQEBP3ULS5BG50LWUCJWSTC"
VERSION = "20180604"
LIMIT = 100

## Creating a function which explore the neighbourhood in downtown Toronto

In [220]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        #create the API request URL
        url = "https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
            CLIENT_ID,
            CLIENT_SECRET,
            VERSION,
            lat,
            lng,
            radius,
            LIMIT)
        
        results = requests.get(url).json()["response"]["groups"][0]["items"]
        
        venues_list.append([(
            name,
            lat,
            lng,
            v["venue"]["name"],
            v["venue"]["location"]["lat"],
            v["venue"]["location"]["lng"],
            v["venue"]["categories"][0]["name"]) for v in results])
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ["Neighborhood",
                    "Neighborhood Latitude",
                    "Neighborhood Longitude",
                    "Venue",
                    "Venue Latitude",
                    "Venue Longitude",
                    "Venue Category"]
    return(nearby_venues)

## Get all nearby benues in neighbourhoods

In [221]:
dToronto_venues = getNearbyVenues(names=Toronto_data["Neighborhood"],
                                    latitudes=Toronto_data["Latitude"],
                                    longitudes=Toronto_data["Longitude"]
                                   )

Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Queen's Park, Ontario Provincial Government


## Getting information about venues

In [222]:
print("There are a total of", dToronto_venues.shape[0], "venues in downtown Toronto grouped in", len(dToronto_venues["Venue Category"].unique()),"categories")
dToronto_venues.head()

There are a total of 1213 venues in downtown Toronto grouped in 210 categories


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.679563,-79.377529,Rosedale Park,43.682328,-79.378934,Playground
1,Rosedale,43.679563,-79.377529,Whitney Park,43.682036,-79.373788,Park
2,Rosedale,43.679563,-79.377529,Alex Murray Parkette,43.678300,-79.382773,Park
3,Rosedale,43.679563,-79.377529,Milkman's Lane,43.676352,-79.373842,Trail
4,"St. James Town, Cabbagetown",43.667967,-79.367675,Cranberries,43.667843,-79.369407,Diner


# Further Analysis of downtown Toronto neighbourhoods

In [223]:
dToronto_spots = pd.get_dummies(dToronto_venues[["Venue Category"]], prefix="", prefix_sep="")

#Then, adding neighbourhood coliumn to dataframe back
dToronto_spots["Neighborhood"] = dToronto_venues["Neighborhood"]

#transfer neighbourhood column to the first one
fixed_columns = [dToronto_spots.columns[-1]] + list(dToronto_spots.columns[:-1])
dToronto_spots = dToronto_spots[fixed_columns]
Group_neighbourhood = dToronto_spots.groupby('Neighborhood').mean().reset_index()

## Top 3 venue's categories per neighborhood

In [224]:
venues = 4
print("***** These are the top 3 venue categories in each neighborhood of downtown Toronto *****")
print("\n")
for hood in Group_neighbourhood["Neighborhood"]:
    print("----"+hood+"----")
    tbl = Group_neighbourhood[Group_neighbourhood["Neighborhood"]==hood].T.reset_index()
    tbl.columns = ["venue", "freq"]
    tbl = tbl.iloc[1:]
    tbl["freq"] = tbl["freq"].astype(float)
    tbl = tbl.round({"freq":2})
    print(tbl.sort_values("freq", ascending=False).reset_index(drop=True).head(venues))
    print("\n")

***** These are the top 3 venue categories in each neighborhood of downtown Toronto *****


----Berczy Park----
          venue  freq
0   Coffee Shop  0.07
1  Cocktail Bar  0.06
2        Bakery  0.04
3          Café  0.04


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0   Airport Service  0.20
1  Airport Terminal  0.13
2     Boat or Ferry  0.07
3          Boutique  0.07


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.17
1  Italian Restaurant  0.06
2      Sandwich Place  0.06
3                Café  0.05


----Christie----
                venue  freq
0       Grocery Store  0.24
1                Café  0.18
2                Park  0.12
3  Athletics & Sports  0.06


----Church and Wellesley----
                 venue  freq
0          Coffee Shop  0.08
1  Japanese Restaurant  0.06
2     Sushi Restaurant  0.06
3           Restaurant  0.04


----Commerce Court, Victo

## The 5 most common venues per neighborhood

### Creating a function with the 5 most common venues

In [225]:
def return_5most_venues(row,n):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:n]

### Apply the factor 8

In [226]:
venues = 8

indicators = ["st", "nd", "rd"]

#below we create the columns according to the number of top venues
columns = ["Neighborhood"]
for ind in np.arange(venues):
    try:
        columns.append("{}{} Most common venue".format(ind+1, indicators[ind]))
    except:
         columns.append("{}th Most common venue".format(ind+1))
            
#next we create a new dataframe
neighborhoods_venues_sort = pd.DataFrame(columns=columns)
neighborhoods_venues_sort["Neighborhood"] = Group_neighbourhood["Neighborhood"]

for ind in np.arange(Group_neighbourhood.shape[0]):
    neighborhoods_venues_sort.iloc[ind, 1:] = return_5most_venues(Group_neighbourhood.iloc[ind, :], venues)
    
neighborhoods_venues_sort.head()

,Neighborhood,1st Most common venue,2nd Most common venue,3rd Most common venue,4th Most common venue,5th Most common venue,6th Most common venue,7th Most common venue,8th Most common venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Cheese Shop,Bakery,Café,Restaurant,Seafood Restaurant,Beer Bar
1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Terminal,Plane,Boat or Ferry,Rental Car Location,Sculpture Garden,Harbor / Marina,Boutique
2,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Japanese Restaurant,Salad Place,Bubble Tea Shop,Thai Restaurant
3,Christie,Grocery Store,Café,Park,Baby Store,Candy Store,Coffee Shop,Restaurant,Athletics & Sports
4,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Gay Bar,Yoga Studio,Café,Men's Store


## Clustering the downtown Toronto Neighborhood

In [227]:
#set the number of the clusters
kclusters = 5

dToronto_clusters = Group_neighbourhood.drop("Neighborhood", 1)

#applying the kmeans clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dToronto_clusters)

kmeans.labels_[0:8]

array([0, 1, 0, 4, 0, 0, 0, 0], dtype=int32)

In [228]:
#adding the labels of clustering
neighborhoods_venues_sort.insert(0, "Cluster Labels", kmeans.labels_)
dToronto_clustered = Toronto_data


dToronto_clustered = dToronto_clustered.join(neighborhoods_venues_sort.set_index("Neighborhood"), on="Neighborhood")
dToronto_clustered.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most common venue,2nd Most common venue,3rd Most common venue,4th Most common venue,5th Most common venue,6th Most common venue,7th Most common venue,8th Most common venue
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,2,Park,Playground,Trail,Cosmetics Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store
1,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675,0,Restaurant,Coffee Shop,Park,Pub,Café,Chinese Restaurant,Pizza Place,Bakery
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,0,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Gay Bar,Yoga Studio,Café,Men's Store
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Pub,Park,Bakery,Breakfast Spot,Café,Theater,Cosmetics Shop
4,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Clothing Store,Coffee Shop,Restaurant,Bubble Tea Shop,Middle Eastern Restaurant,Café,Japanese Restaurant,Italian Restaurant


## Visualizing clusters on the map

In [239]:
toronto_cluster_map =folium.Map(location=[TorLat,TorLong], zoom_start=14)

x =np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0,1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
dToronto_clustered['Cluster Labels']
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dToronto_clustered['Latitude'], dToronto_clustered['Longitude'], dToronto_clustered['Neighborhood'], dToronto_clustered['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.6).add_to(toronto_cluster_map)
       
toronto_cluster_map

## Exploring each cluster

In [240]:
Cluster0 = dToronto_clustered.loc[dToronto_clustered['Cluster Labels'] == 0,
                                  dToronto_clustered.columns[[1] + list(range(6, dToronto_clustered.shape[1]))]]
Cluster0

,Borough,1st Most common venue,2nd Most common venue,3rd Most common venue,4th Most common venue,5th Most common venue,6th Most common venue,7th Most common venue,8th Most common venue
1,Downtown Toronto,Restaurant,Coffee Shop,Park,Pub,Café,Chinese Restaurant,Pizza Place,Bakery
2,Downtown Toronto,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Gay Bar,Yoga Studio,Café,Men's Store
3,Downtown Toronto,Coffee Shop,Pub,Park,Bakery,Breakfast Spot,Café,Theater,Cosmetics Shop
4,Downtown Toronto,Clothing Store,Coffee Shop,Restaurant,Bubble Tea Shop,Middle Eastern Restaurant,Café,Japanese Restaurant,Italian Restaurant
5,Downtown Toronto,Café,Coffee Shop,Cocktail Bar,American Restaurant,Gastropub,Gym,Beer Bar,Clothing Store
6,Downtown Toronto,Coffee Shop,Cocktail Bar,Cheese Shop,Bakery,Café,Restaurant,Seafood Restaurant,Beer Bar
7,Downtown Toronto,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Japanese Restaurant,Salad Place,Bubble Tea Shop,Thai Restaurant
8,Downtown Toronto,Coffee Shop,Café,Restaurant,Hotel,Deli / Bodega,Thai Restaurant,Clothing Store,Gym
9,Downtown Toronto,Coffee Shop,Aquarium,Café,Hotel,Restaurant,Sporting Goods Shop,Scenic Lookout,Italian Restaurant
10,Downtown Toronto,Coffee Shop,Hotel,Café,Restaurant,Salad Place,Japanese Restaurant,Seafood Restaurant,Italian Restaurant


In [241]:
Cluster1 = dToronto_clustered.loc[dToronto_clustered['Cluster Labels'] == 1,
                                  dToronto_clustered.columns[[1] + list(range(6, dToronto_clustered.shape[1]))]]
Cluster1

,Borough,1st Most common venue,2nd Most common venue,3rd Most common venue,4th Most common venue,5th Most common venue,6th Most common venue,7th Most common venue,8th Most common venue
14,Downtown Toronto,Airport Service,Airport Terminal,Plane,Boat or Ferry,Rental Car Location,Sculpture Garden,Harbor / Marina,Boutique


In [242]:
Cluster2 = dToronto_clustered.loc[dToronto_clustered['Cluster Labels'] == 2,
                                  dToronto_clustered.columns[[1] + list(range(6, dToronto_clustered.shape[1]))]]
Cluster2

,Borough,1st Most common venue,2nd Most common venue,3rd Most common venue,4th Most common venue,5th Most common venue,6th Most common venue,7th Most common venue,8th Most common venue
0,Downtown Toronto,Park,Playground,Trail,Cosmetics Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store


In [243]:
Cluster3 = dToronto_clustered.loc[dToronto_clustered['Cluster Labels'] == 3,
                                  dToronto_clustered.columns[[1] + list(range(6, dToronto_clustered.shape[1]))]]
Cluster3

,Borough,1st Most common venue,2nd Most common venue,3rd Most common venue,4th Most common venue,5th Most common venue,6th Most common venue,7th Most common venue,8th Most common venue
12,Downtown Toronto,Café,Bar,Italian Restaurant,Japanese Restaurant,Bookstore,Restaurant,Bakery,Yoga Studio


In [244]:
Cluster4 = dToronto_clustered.loc[dToronto_clustered['Cluster Labels'] == 4,
                                  dToronto_clustered.columns[[1] + list(range(6, dToronto_clustered.shape[1]))]]
Cluster4

,Borough,1st Most common venue,2nd Most common venue,3rd Most common venue,4th Most common venue,5th Most common venue,6th Most common venue,7th Most common venue,8th Most common venue
17,Downtown Toronto,Grocery Store,Café,Park,Baby Store,Candy Store,Coffee Shop,Restaurant,Athletics & Sports


## Naming clusters

In [262]:
Cluster_Name = pd.DataFrame(columns = ['Cluster num', 'Cluster Labels'])
Cluster_Name['Cluster num'] = dToronto_clustered['Cluster Labels']
test = []
clusName = []
for j in range(5):
    t = []
    Clus = dToronto_clustered.loc[dToronto_clustered['Cluster Labels'] == j, dToronto_clustered.columns[[1] + 
list(range(6, dToronto_clustered.shape[1]))]]
    for i in range(1,5):
        t.append(Clus.loc[0:,Cluster0.columns[i]].mode()[0]) 
    clusName = list(set(t[1:]))
    #Cluster_Name.loc(dToronto_clustered['Cluster Labels'],'Cluster name') = str(clusNames)
    print(clusName)
    test.append(clusName)

for k in range(5):
    Cluster_Name.loc[dToronto_clustered['Cluster Labels'] == k, 'Cluster Labels'] = ", ".join(test[k])

['Hotel', 'Restaurant', 'Café']
['Plane', 'Boat or Ferry', 'Airport Terminal']
['Cosmetics Shop', 'Trail', 'Playground']
['Bar', 'Japanese Restaurant', 'Italian Restaurant']
['Park', 'Baby Store', 'Café']


In [263]:
Cluster_Name

,Cluster num,Cluster Labels
0,2,"Cosmetics Shop, Trail, Playground"
1,0,"Hotel, Restaurant, Café"
2,0,"Hotel, Restaurant, Café"
3,0,"Hotel, Restaurant, Café"
4,0,"Hotel, Restaurant, Café"
5,0,"Hotel, Restaurant, Café"
6,0,"Hotel, Restaurant, Café"
7,0,"Hotel, Restaurant, Café"
8,0,"Hotel, Restaurant, Café"
9,0,"Hotel, Restaurant, Café"


## Visualizing the map with new named clusters

In [265]:
new_map_clusters = folium.Map(location=[TorLat, TorLong], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
dToronto_clustered['Cluster Labels']
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster, k in zip(dToronto_clustered['Latitude'], dToronto_clustered['Longitude'], dToronto_clustered['Neighborhood'], Cluster_Name['Cluster Labels'], Cluster_Name['Cluster num']):
    label = folium.Popup('Neighborhood: ' + str(poi) + '. Venues: ' + cluster, parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[k-1],
        fill=True,
        fill_color=rainbow[k-1],
        fill_opacity=0.6).add_to(new_map_clusters)
       
new_map_clusters